## Import and Set

In [1]:
import sys 
sys.path.append('../')
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from utils import *
from datas import *
from set_train import *
from models.Transformers import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
stock_symbol, end_date, num_class, batch_size, init, fp16_training, num_epochs, lr = set_train()

# Data
trainloader, validloader, testloader, test_date, df, src = data(num_class=num_class, batch_size=batch_size)
for x, y in trainloader:
    break
print(src.device, x.device, src.shape, x.shape, y.shape)

100%|██████████| 2965/2965 [00:03<00:00, 813.81it/s] 

x_train_len: 2128
cpu cpu torch.Size([2000, 6, 1]) torch.Size([128, 6, 10]) torch.Size([128, 2])


## Init: Model, Criteria, Optimizer, Fp16, Previous Tarin Inofrmation

In [2]:
"""
Choose if fp16 and define model
pip install accelerate==0.2.0
"""
# Model
if fp16_training:
    from accelerate import Accelerator
    print('Accelerating')
    accelerator = Accelerator()
    device = accelerator.device
    model = TransformerEncoderDecoder(num_class=num_class)
else:
    model = TransformerEncoderDecoder(num_class=num_class).to(device)
        
Model = model.model_type # Model name

"""
Init for models, learning rate, ...
"""
# Check path
if os.path.exists(f'Temp//{Model}_{stock_symbol}_LastTrainInfo.pk'):
    if init:
        print("Init model")
        lr = lr
        last_epoch = 0
        min_val_loss = 10000
        loss_train = []
        loss_valid = []
    else:
        print('Load from last train epoch')
        model.load_state_dict(torch.load(f'Temp//{Model}_class{num_class}_{stock_symbol}_checkpoint_LastTrainModel.pt'))
        with open(f'Temp//{Model}_class{num_class}_{stock_symbol}_LastTrainInfo.pk', 'rb') as f:
            last_train_info = pickle.load(f)
        with open(f'Temp//{Model}_class{num_class}_{stock_symbol}_TrainValHistLoss.pk', 'rb') as f:
            loss_train_val = pickle.load(f)            
        lr = last_train_info['lr']
        last_epoch = last_train_info['epoch']
        min_val_loss = last_train_info['min val loss']
        loss_train = loss_train_val['train']
        loss_valid = loss_train_val['valid']
else:
        print("Init model")
        lr = lr
        last_epoch = 0
        min_val_loss = 10000.0
        loss_train = []
        loss_valid = []
        
print(
    f'Last train epoch: {last_epoch}  '
    f'Last train lr: {lr}   '
    f'Min val loss: {min_val_loss}'
    )

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.00001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)        

if fp16_training:
    print('Accelerate Prepare')    
    model, optimizer, trainloader, validloader, scheduler = \
    accelerator.prepare(model, optimizer, trainloader, validloader, scheduler)
        
for name, param in model.named_parameters():
    print(f"Parameter '{name}' is on device: {param.device}")
    break

Accelerating
Init model
Last train epoch: 0  Last train lr: 0.01   Min val loss: 10000.0
Accelerate Prepare
Parameter 'embedding.weight' is on device: cuda:0


## Train

In [3]:
Model

'TransEnDecoder-Window10EL512DL16Hid512'

In [4]:
"""
--- Original ---------
batch_x: (batch_size, d_model, seqlen) 
src: (total_length, d_model, seq_len)
--- Input of model ---
batch_x: (batch_size, seq_len, d_model) -> use src.permute()
src: (total_length, seq_len, d_model)   -> use batch.permute()
"""
src = src.squeeze(2).unsqueeze(0).to(device)
for epoch in range(last_epoch, num_epochs):
    # Training
    model.train()
    loss_train_e = 0
    for batch_x, batch_y in tqdm(trainloader): 
        # fp16
        if not fp16_training:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)            
        # Permute
        batch_x = batch_x.permute(0, 2, 1)        
        # Zero grad
        optimizer.zero_grad()        
        # Drop last batch
        if batch_x.size(0) != batch_size:
            continue        
        # Model
        memory, outputs = model(src=src, tgt=batch_x, train=True)    
        # Loss
        loss = criterion(outputs, batch_y)
        if fp16_training:
            accelerator.backward(loss)
        else:
            loss.backward()
        optimizer.step()
        loss_train_e += loss.item()
    
    # Train loss
    loss_train_e /= len(trainloader)
    loss_train.append(loss_train_e)
    
    # Scheduler 
    if epoch > 10:
        scheduler.step()
    
    # Validating
    loss_valid_e = 0
    with torch.no_grad():
        model.eval()
        for batch_x_val, batch_y_val in tqdm(validloader):
            # batch_x_val = mask(batch_x_val)
            if not fp16_training:
                batch_x_val = batch_x_val.to(device)
                batch_y_val = batch_y_val.to(device)
            batch_x_val = batch_x_val.permute(0, 2, 1)
            
            if batch_x_val.size(0) != batch_size:
                    continue
            memory, outputs_val = model(src, batch_x_val, False, memory)
                
            loss = criterion(outputs_val, batch_y_val)
            loss_valid_e += loss.item()
        loss_valid_e /= len(validloader)
        loss_valid.append(loss_valid_e)
            
        torch.save(model.state_dict(), f'Temp/{Model}_class{num_class}_{stock_symbol}_checkpoint_LastTrainModel.pt')
        if loss_valid_e < min_val_loss:
            min_val_loss = loss_valid_e
            print(f'New best model found in epoch {epoch} with val loss: {min_val_loss}')
            torch.save(model.state_dict(), f'Model_Result/{Model}_class{num_class}_{stock_symbol}_best_model.pt')            
        if epoch % 50 == 0:
            pass
            # torch.save(model, f'ConformerResult/Conformerr_{stock_symbol}_checkpoint_{epoch}.pt')
     
    with open(f'Temp/{Model}_class{num_class}_{stock_symbol}_TrainValHistLoss.pk', 'wb') as f:
        pickle.dump({'train': loss_train, 'valid': loss_valid}, f)
    with open(f'Temp/{Model}_class{num_class}_{stock_symbol}_LastTrainInfo.pk', 'wb') as f:
        pickle.dump({'min val loss': min_val_loss, 'epoch': epoch, 'lr': optimizer.param_groups[0]['lr']}, f)        
    print(
        f'Epoch [{epoch}/{num_epochs}]',
        f'Training Loss: {loss_train_e:.5f}',
        f'Valid Loss: {loss_valid_e:.5f}'
        )

100%|██████████| 4/4 [00:00<00:00,  4.63it/s]


Epoch [42/50] Training Loss: 4.19455 Valid Loss: 5.21695


100%|██████████| 4/4 [00:00<00:00,  4.63it/s]


Epoch [43/50] Training Loss: 4.26757 Valid Loss: 5.21657


 25%|██▌       | 4/16 [00:22<01:08,  5.72s/it]